In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

import glob
import random
import tensorflow as tf
from shutil import rmtree
from keras.models import load_model
from os.path import join, getctime, basename
from models import *
from utils import *
from data_preprocess import load_prep_data

%load_ext autoreload
%autoreload 2

aug_data = load_prep_data(time=1000, debug=False, pooling = False, subsample = 1, average = 1, noise_level = 0)
x_train = aug_data['x_train']
y_train = aug_data['y_train']
x_valid = aug_data['x_valid']
y_valid = aug_data['y_valid']
x_test = aug_data['x_test']
y_test = aug_data['y_test']

input_shape = x_train.shape #（6960, 22, 250)
x_train = tf.transpose( tf.expand_dims(x_train, axis=-1), perm=[0, 2, 3, 1])
x_valid = tf.transpose( tf.expand_dims(x_valid, axis=-1), perm=[0, 2, 3, 1])

print("x_train.shape: {}", format(x_train.shape))

x_train.shape: {} (1798, 1000, 1, 22)


In [2]:
config = {
    # Network
    'num_inputs': x_train.shape[0],
    'input_shape': (input_shape[2], 1, input_shape[1]),
    'epochs': 100,
    'dropout': 0.5,
    'batch_size': 32
}
CNN4LayerGRU = CNN4LayerGRU()
CNN4LayerGRU.build_model(config)
history = CNN4LayerGRU.train(x_train, y_train, x_valid, y_valid, config, get_workpath('CNN4LayerGRU'))

x_test_transposed = tf.transpose( tf.expand_dims(x_test, axis=-1), perm=[0, 2, 3, 1])
raw = CNN4LayerGRU.evaluate(x_test_transposed, y_test)
print("Raw Acc result: {}".format(raw[1]))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1000, 1, 25)       13775     
                                                                 
 max_pooling2d (MaxPooling2  (None, 334, 1, 25)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 334, 1, 25)        100       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 334, 1, 25)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 334, 1, 50)        31300     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 112, 1, 50)        0